In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
#Leer
df = pd.read_csv('movies_dataset_clean.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41460 entries, 0 to 41459
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             41460 non-null  int64  
 1   genres             41035 non-null  object 
 2   id                 41460 non-null  int64  
 3   original_language  41454 non-null  object 
 4   popularity         41457 non-null  float64
 5   release_date       41460 non-null  object 
 6   revenue            41460 non-null  float64
 7   runtime            41441 non-null  float64
 8   title              41460 non-null  object 
 9   vote_average       41457 non-null  float64
 10  vote_count         41457 non-null  float64
 11  languages          38014 non-null  object 
 12  director           40739 non-null  object 
 13  overview_clean     41460 non-null  object 
 14  release_year       41459 non-null  float64
 15  return             41459 non-null  float64
dtypes: float64(7), int64(2

In [5]:
#MOSTRAR PELICULAS CON TITULOS IGUALES

duplicados = df[df.duplicated(['title'], keep=False)]
duplicados = duplicados.sort_values('title')

duplicados[['title', 'vote_count', 'release_year']]

,title,vote_count,release_year
1161,12 Angry Men,2130.0,1957.0
15194,12 Angry Men,59.0,1997.0
32753,12 Chairs,16.0,1976.0
32767,12 Chairs,24.0,1971.0
990,"20,000 Leagues Under the Sea",160.0,1954.0
...,...,...,...
11643,Zodiac,2080.0,2007.0
11168,Zoom,140.0,2006.0
37870,Zoom,25.0,2015.0
24646,Zulu,176.0,2013.0


MODELO 1

In [6]:
# Seleccionando columnas necesarias
modelo1 = df[['title', 'overview_clean', 'genres', 'director']].copy()
#preprocesamiento "genres" para separar géneros y convertir en palabras individuales
modelo1['genres'] = modelo1['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
#preprocesamiento en "director" para convertirlos en palabras individuales
modelo1['director'] = modelo1['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))


In [7]:
# crea instancia de TfidfVectorizer con parámetros deseados
tfidf_1 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar TF-IDF a "overview_clean", "genres" y "director"
tfidf_matriz_1 = tfidf_1.fit_transform(modelo1['overview_clean'] + ' ' + modelo1['genres'] + ' ' + modelo1['director'])



In [8]:
# Cantidad de registros
tfidf_matriz_1.shape

(41460, 1004258)

In [9]:
def get_recomendacion_m1(titulo):
  indices = pd.Series(modelo1.index, index=modelo1['title']).drop_duplicates()
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  if idx is None:
    return "Película no encontrada"
  if modelo1.duplicated(['title']).any():
    primer_idx = modelo1[modelo1['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  #similitud coseno entre película de entrada y demás películas
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_1[idx], tfidf_matriz_1).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  recomendaciones = modelo1.iloc[[i[0] for i in simil], :]['title'].tolist()

  return recomendaciones


In [10]:
# Prueba 1
get_recomendacion_m1('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman Returns',
 'Batman',
 'Batman Begins']

In [11]:
# Prueba 2
get_recomendacion_m1('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Star Wars: The Force Awakens',
 'Star Wars: Episode I - The Phantom Menace']

In [12]:
# Prueba 3
get_recomendacion_m1('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'Indie Game: The Movie']

MODELO 2

In [13]:
# Seleccionar columnas necesarias para el modelo
modelo2 = df[['genres', 'title', 'vote_count', 'vote_average', 'director']].copy()


In [14]:
# Procesar los datos
modelo2['genres'] = modelo2['genres'].fillna('')
modelo2['genres'] = modelo2['genres'].str.split(',')
genres = pd.Series([genre for genres in modelo2['genres'] for genre in genres]).unique()
for genre in genres:
  modelo2[genre] = modelo2['genres'].apply(lambda x: 1 if genre in x else 0)
# nueva columna con los nombres de los directores
modelo2['directores'] = modelo2['director'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])


In [15]:
# Calificación promedio
v = modelo2['vote_count']
# Número de votos
R = modelo2['vote_average']
# Número mínimo de votos requerido
m = 25
# calificación promedio de todas las peliculas
C = modelo2['vote_average'].mean()
# Calificacion ponderada de películas y agregarla como una nueva columna
modelo2['calificacion_ponderada'] = ((v / (v + m)) * R) + ((m / (v + m)) * C)


In [16]:
modelo2.head()

,genres,title,vote_count,vote_average,director,animation,comedy,family,adventure,fantasy,...,war,foreign,music,documentary,western,tvmovie,,short,directores,calificacion_ponderada
0,"[animation, comedy, family]",Toy Story,5415.0,7.7,John Lasseter,1,1,1,0,0,...,0,0,0,0,0,0,0,0,[John Lasseter],7.690553
1,"[adventure, fantasy, family]",Jumanji,2413.0,6.9,Joe Johnston,0,0,1,1,1,...,0,0,0,0,0,0,0,0,[Joe Johnston],6.887124
2,"[romance, comedy]",Grumpier Old Men,92.0,6.5,Howard Deutch,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Howard Deutch],6.317155
3,"[comedy, drama, romance]",Waiting to Exhale,34.0,6.1,Forest Whitaker,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Forest Whitaker],5.906900
4,[comedy],Father of the Bride Part II,173.0,5.7,Charles Shyer,0,1,0,0,0,...,0,0,0,0,0,0,0,0,[Charles Shyer],5.692965


In [17]:
# Función para recomendar peliculas
def get_recomendacion_m2(titulo):
  pelicula = modelo2.loc[modelo2['title'] == titulo].iloc[0]
  # Calcular similitud
  modelo2['similitud'] = modelo2.apply(lambda x: (x[genres] * pelicula[genres]).sum() / (x[genres].sum() + pelicula[genres].sum()), axis=1)
  # Ordenar por similitud y obtener 5 mejores
  recomendaciones = modelo2.sort_values('similitud', ascending=False).head(6)[1:]
  # Calcular peso adicional para peliculas con calificacion ponderada alta y agregarlo a recomendaciones
  recomendaciones['peso_calificacion'] = recomendaciones['calificacion_ponderada'].apply(lambda x: 1 if x >= pelicula['calificacion_ponderada'] else 0)
  # Calcular peso adicional para peliculas con mismo director que la pelocula recomendanda y agregarlo a las recomendaciones
  recomendaciones['peso_director'] = recomendaciones.apply(lambda x: len(set(x['directores']).intersection(set(pelicula['directores']))) /
                                                           len(set(x['directores']).union(set(pelicula['directores']))), axis=1)
  # puntuación total para cada recomendacion
  recomendaciones['peso_total'] = recomendaciones['similitud'] + (recomendaciones['peso_calificacion'] * 0.2) + (recomendaciones['peso_director'] * 0.2)
  recomendaciones = recomendaciones.sort_values('peso_total', ascending=False).head(5)

  return recomendaciones['title'].tolist()

In [18]:
# Prueba #1
get_recomendacion_m2('The Dark Knight Rises')

['Shinjuku Incident',
 'Colt 45',
 'The Grissom Gang',
 'Rolling Thunder',
 'After The Storm']

In [19]:
# Prueba #2
get_recomendacion_m2('Wonder Woman')

['Wonder Woman',
 'Ultimate Avengers',
 'Justice League: The New Frontier',
 'Green Lantern: Emerald Knights',
 'Macross Plus: Movie Edition']

EVALUACION

In [20]:
def evaluar_precision(modelo, titulos_prueba, recomendaciones_esperadas, k=5):

    true_positives = 0
    total_relevantes = 0
    total_recomendadas = 0

    for titulo in titulos_prueba:
        if titulo in recomendaciones_esperadas:
            # Obtener las recomendaciones del modelo
            recomendaciones = modelo(titulo)[:k]
            relevantes = set(recomendaciones_esperadas[titulo])
            recomendadas = set(recomendaciones)

            # Calcular métricas
            true_positives += len(relevantes.intersection(recomendadas))
            total_relevantes += len(relevantes)
            total_recomendadas += len(recomendadas)

    # Calcular precision@k y recall@k
    precision_k = true_positives / total_recomendadas if total_recomendadas > 0 else 0
    recall_k = true_positives / total_relevantes if total_relevantes > 0 else 0

    return {'precision@k': precision_k, 'recall@k': recall_k}


In [21]:
import time
import numpy as np

def evaluar_tiempo_promedio(modelo, titulos_prueba):

    tiempos = []
    for titulo in titulos_prueba:
        inicio = time.time()  # Inicia temporizador
        modelo(titulo)        # Llama al modelo con un título
        fin = time.time()     # Detiene temporizador
        tiempos.append(fin - inicio)  # Calcula duración

    return np.mean(tiempos)  # Devuelve el tiempo promedio


In [ ]:
# Definición de títulos de prueba y recomendaciones esperadas
titulos_prueba = ['The Dark Knight Rises', 'Star Wars', 'The Hunger Games']
recomendaciones_esperadas = {
    'The Dark Knight Rises': ['The Dark Knight', 'Batman Begins', 'Inception'],
    'Star Wars': ['Star Wars: Episode V - The Empire Strikes Back', 'Star Wars: Episode IV - A New Hope'],
    'The Hunger Games': ['The Hunger Games: Catching Fire', 'Divergent', 'The Maze Runner']
}

# Evaluar precisión y recall para el Modelo 1
precision_modelo1 = evaluar_precision(get_recomendacion_m1, titulos_prueba, recomendaciones_esperadas, k=5)
print("Modelo 1 - Precision:", precision_modelo1)

# Evaluar tiempo promedio para el Modelo 1
tiempo_modelo1 = evaluar_tiempo_promedio(get_recomendacion_m1, titulos_prueba)
print("Modelo 1 - Tiempo promedio:", tiempo_modelo1, "segundos")

# Evaluar precisión y recall para el Modelo 2
precision_modelo2 = evaluar_precision(get_recomendacion_m2, titulos_prueba, recomendaciones_esperadas, k=5)
print("Modelo 2 - Precision:", precision_modelo2)

# Evaluar tiempo promedio para el Modelo 2
tiempo_modelo2 = evaluar_tiempo_promedio(get_recomendacion_m2, titulos_prueba)
print("Modelo 2 - Tiempo promedio:", tiempo_modelo2, "segundos")


Modelo 1 - Precision: {'precision@k': 0.2, 'recall@k': 0.375}
Modelo 1 - Tiempo promedio: 0.1767857869466146 segundos
Modelo 2 - Precision: {'precision@k': 0.0, 'recall@k': 0.0}
